In [1]:
import subprocess
# from extract_spkr_emb import get_embedding
import librosa
import torch
import os
from nemo.core import typecheck
import numpy as np

## Creating the JIT

In [ ]:
from nemo.collections.asr.models import EncDecSpeakerLabelModel
speaker_encoder = EncDecSpeakerLabelModel.from_pretrained(model_name="ecapa_tdnn").cuda()

speaker_encoder.eval()

ref_path = "miipherTestDataset/train/clean_trainset_wav/p234_002.wav"
audio, sr = librosa.load(ref_path, sr=16000)
audio = np.array([audio])
audio_length = audio.shape[-1]
audio_signal, audio_signal_len = (
    torch.tensor(audio),
    torch.tensor([audio_length])
)

audio_signal.shape

typecheck.set_typecheck_enabled(False)

kwarg = {'input_signal': audio_signal, 'input_signal_length': audio_signal_len}

# kwarg = {'input_signal': ref_waveform, 'input_signal_length': length}
with torch.no_grad():
    traced_model = torch.jit.trace(speaker_encoder, example_inputs=(kwarg['input_signal'].cuda(),kwarg['input_signal_length'].cuda()))
    torch.jit.save(traced_model, f"ecapa2_traced.jit")

In [12]:
kwarg = {'input_signal': audio_signal.cuda(), 'input_signal_length': audio_signal_len.cuda()}

m = torch.jit.load("ecapa2_traced.jit")

m(**kwarg)[-1] #logits, embedding

tensor([[-8.3511e-03,  3.7691e-02,  4.0074e-02,  1.7123e-03,  2.1409e-02,
          1.8274e-02,  3.4582e-02, -3.1881e-02,  2.8568e-02,  1.0042e-02,
         -7.6076e-03, -8.4978e-02,  6.5006e-02,  9.5014e-03,  4.3389e-02,
          7.9106e-03, -9.7719e-03,  2.3300e-02, -1.9371e-02,  3.3867e-03,
         -1.9562e-02,  1.3623e-02, -4.7087e-02,  4.1147e-02,  3.0104e-03,
         -2.3696e-02, -2.0475e-02,  6.9882e-03, -2.4294e-02,  4.1677e-02,
          9.1270e-03, -2.2799e-02,  1.8357e-02,  5.1804e-02, -7.0486e-04,
          1.7024e-03,  4.6128e-02,  1.6992e-02,  4.7238e-02, -1.9965e-02,
         -1.1293e-02, -5.6970e-02, -1.5580e-02,  5.3834e-03,  5.1846e-02,
         -2.8371e-02,  1.7918e-02, -2.7423e-02, -2.6831e-02,  6.2204e-02,
         -2.6433e-02,  3.1800e-02, -1.9426e-02, -2.4741e-02,  2.9004e-02,
         -8.9724e-05, -8.5246e-03,  4.2694e-02,  4.1995e-03,  1.4181e-02,
          6.9114e-02, -6.6858e-03, -8.1257e-03,  3.2910e-03, -3.0386e-02,
          4.9475e-02,  4.2692e-03, -2.

## Scores

In [2]:
root = 'testing'
root2 = "tests"
inputs = ['malea1','malea2','maleb1','maleb2','femalea1','femalea2','femaleb1','femaleb2']
test1 = ['maleadc','malebdc','femaleadc','femalebdc']
test2 = ['brandon','camille']
test3 = ['maleu1c','maleu2c','femaleu1c','femaleu2c']

In [3]:
embs = {}
for x in test1+test2+test3:
    ref_waveform, sr = librosa.load(f"{root}/{x}.wav", sr=16000)
    embs[x] = get_embedding(torch.from_numpy(ref_waveform))

In [4]:
def get_score(embs1, embs2):
    #cand1/2 = (ith speaker, jth segment)
    X = embs1
    Y = embs2
    # Length Normalize
    # X = embs1 / torch.linalg.norm(embs1)
    # Y = embs2 / torch.linalg.norm(embs2)
    # Score
    similarity_score = torch.dot(X, Y) / ((torch.dot(X, X) * torch.dot(Y, Y)))
    # similarity_score = (similarity_score + 1) / 2
    return similarity_score.item()

In [5]:
scores = {}
for file in os.listdir("tests/"):
    if "wav" in file:
        filePath = f"{root2}/{file}"
        name = os.path.splitext(file)[0]
        compare_emb = embs[name.split("-")[-1]]
        ref_waveform, sr = librosa.load(filePath, sr=16000)
        query_emb = get_embedding(torch.from_numpy(ref_waveform))
        scores[file] = get_score(query_emb.flatten(), compare_emb.flatten())

In [11]:
with open('scores.csv', 'w') as f:
    f.write("filename, score\n")
    for key in scores.keys():
        f.write("%s,%s\n"%(key,scores[key]))

In [23]:
import IPython.display as ipd
ref_waveform, sr = librosa.load("/Users/ajaybati/Downloads/0.23.0612.1.AT.PHL.alorica/results/output_files/malea1-camille.wav", sr=16000)
ref_waveform2, sr2 = librosa.load("/Users/ajaybati/Downloads/0.23.0612.1.AT.PHL.alorica/results/originalTestFiles/camille.wav", sr=16000)
ref_waveform3, sr3 = librosa.load("/Users/ajaybati/Downloads/0.23.0612.1.AT.PHL.alorica/results/originalTestFiles/malea1.wav", sr=16000)

ipd.Audio(ref_waveform, rate=sr)

In [24]:
ipd.Audio(ref_waveform2, rate=sr2)

In [25]:
ipd.Audio(ref_waveform3, rate=sr3)

In [26]:
keys = scores.keys()
dicts = [{},{},{}]
for x in keys:
    name = os.path.splitext(x)[0]
    compare = name.split("-")[-1]
    if compare in test1:
        dicts[0][x] = scores[x]
    elif compare in test2:
        dicts[1][x] = scores[x]
    elif compare in test3:
        dicts[2][x] = scores[x]

In [17]:
for x in range(3):
    d = dicts[x]
    with open(f'scores_test{x+1}.csv', 'w') as f:
        f.write("filename, score\n")
        for key in d.keys():
            f.write("%s,%s\n"%(key,d[key]))